<a href="https://colab.research.google.com/github/AnkeHooliganke/bases/blob/main/pandas_vs_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
import pandas as pd

# Reading data into dataframe tables

In [ ]:
employee = pd.read_csv("queries_data/employee.csv")
employee.head()

,ssn,fname,lname,sex,salary,super_ssn,dno
0,1,peter,skachovsky,m,1000,2.0,10
1,2,ronald,dehaan,m,2000,4.0,10
2,3,alina,petrova,f,1500,2.0,10
3,4,sergey,paramonov,m,4000,NaN,20
4,5,andra,busoniu,f,2500,NaN,30


In [ ]:
works_on = pd.read_csv("queries_data/works_on.csv")
works_on.head()

,essn,pno,hours
0,1,1,10
1,1,2,20
2,1,3,5
3,1,4,10
4,1,5,3


In [ ]:
project = pd.read_csv("queries_data/project.csv")
project.head()

,pname,pnumber,plocation,dnum
0,A,1,Dresden,10
1,A,2,Bolzano,10
2,C,3,Moscow,20
3,X,4,Vienna,30
4,Z,5,Houston,20


# Good old queries but now in pandas!

The same problems but in a different paradigm

# Пишем старые добрые запросы, но уже ручками на pandas 

В общем-то те же запросы, что мы видели на реляционной алгебре, реляционном исчислении и SQL -- но теперь на pandas!

## Question 1
Question: Retrieve the names of all employees in department 5 who work more than 10 hours per week on the ProductX project!

SQL code to guide:
``` SQL
SELECT Fname, Lname
FROM EMPLOYEE, WORKS_ON, PROJECT 
WHERE Ssn=Essn
AND Pno=Pnumber
AND Dno=10
AND Hours>=10
AND Pname=’X’
```

In [ ]:
merged_df = pd.merge(left=employee, right=works_on, left_on='ssn', right_on='essn')
merged_df.query("dno==5 & pno==1 & hours > 10")
merged_df[['fname', 'lname']]

,fname,lname
0,peter,skachovsky
1,peter,skachovsky
2,peter,skachovsky
3,peter,skachovsky
4,peter,skachovsky
5,ronald,dehaan
6,ronald,dehaan
7,alina,petrova
8,sergey,paramonov


## Question 2
Question: List the names of all employees who have a
dependent with the same first name as themselves

SQL code to guide:
``` SQL
SELECT Fname, Lname
FROM EMPLOYEE, DEPENDENT
WHERE Ssn=Essn
AND Dependent_name=Fname
```

In [ ]:
dependent = pd.read_csv("queries_data/dependent.csv")
dependent.head()

,essn,dependent_name,sex
0,1,anna,f
1,2,ronald,m
2,3,evgeny,m


In [ ]:
emp_with_dependents = pd.merge(left=employee, right=dependent, left_on='ssn', right_on='essn', how='right')
emp_with_dependents.query("fname == dependent_name")

,ssn,fname,lname,sex_x,salary,super_ssn,dno,essn,dependent_name,sex_y
1,2,ronald,dehaan,m,2000,4.0,10,2,ronald,m


## Question 3
Question: Find the names of all employees who are directly
supervised by ronald dehaan


SQL code to guide:
``` SQL
SELECT E.Fname, E.Lname
FROM EMPLOYEE E, EMPLOYEE S
WHERE E.Super_ssn=S.Ssn
AND S.Fname=’ronald’
AND S.Lname=’dehaan’ 
```

In [ ]:
emp_with_ron = pd.merge(left=employee, right=employee.query("fname == 'ronald' and lname == 'dehaan'"), left_on='super_ssn', right_on='ssn')
emp_with_ron

,ssn_x,fname_x,lname_x,sex_x,salary_x,super_ssn_x,dno_x,ssn_y,fname_y,lname_y,sex_y,salary_y,super_ssn_y,dno_y
0,1,peter,skachovsky,m,1000,2.0,10,2,ronald,dehaan,m,2000,4.0,10
1,3,alina,petrova,f,1500,2.0,10,2,ronald,dehaan,m,2000,4.0,10


In [ ]:
output = ...

## Question  4
Question: For each project, list the project name and the total hours per week (by all employees) spent on that project

SQL code to guide:
``` SQL
SELECT Pno, Pname, SUM(Hours)
FROM PROJECT JOIN WORKS_ON ON Pno=Pnumber 
GROUP BY Pno, Pname
```

In [ ]:
prj_works_on = works_on.groupby('pno')
prj_works_on.describe()


essn                            ... hours                          
    count mean       std  min   25%  ...   min    25%   50%    75%   max
pno                                  ...                                
1     2.0  1.5  0.707107  1.0  1.25  ...  10.0  10.00  10.0  10.00  10.0
2     2.0  2.0  1.414214  1.0  1.50  ...  10.0  12.50  15.0  17.50  20.0
3     1.0  1.0       NaN  1.0  1.00  ...   5.0   5.00   5.0   5.00   5.0
4     2.0  2.5  2.121320  1.0  1.75  ...  10.0  17.50  25.0  32.50  40.0
5     2.0  1.5  0.707107  1.0  1.25  ...   3.0   4.75   6.5   8.25  10.0

[5 rows x 16 columns]

In [ ]:
prj = pd.merge(left=project[['pname', 'pnumber']], right=works_on, left_on='pnumber', right_on='pno').groupby('pno')[['hours']].sum()
prj.head()

,hours
pno,
1,20
2,30
3,5
4,50
5,13


## Question 5
Question: Retrieve the names of all employees who work on every project

SQL code to guide:
``` SQL
SELECT Fname, Lname FROM EMPLOYEE
WHERE NOT EXISTS
    (SELECT * FROM PROJECT
     WHERE NOT EXISTS (SELECT *
                       FROM WORKS_ON
                       WHERE Pno = Pnumber
                       AND Essn = Ssn
                      )
    )
```

In [ ]:
pcnt = pd.unique(project['pnumber']).size
all_p = pd.merge(employee[['fname', 'lname', 'ssn']], works_on, left_on='ssn', right_on='essn')
all_p = pd.merge(left=all_p, right=all_p.groupby('ssn')['pno'].count(), left_on='ssn', right_on='ssn')
all_p = all_p.loc[all_p["pno_y"] == pcnt]
all_p

,fname,lname,ssn,essn,pno_x,hours,pno_y
0,peter,skachovsky,1,1,1,10,5
1,peter,skachovsky,1,1,2,20,5
2,peter,skachovsky,1,1,3,5,5
3,peter,skachovsky,1,1,4,10,5
4,peter,skachovsky,1,1,5,3,5


## Question 6
Question: Retrieve the average salary of all female employees

SQL code to guide:
``` SQL
SELECT AVG(Salary) 
FROM EMPLOYEE 
WHERE Sex=’F’
```

In [ ]:
employee.query("sex == 'm'")['salary'].mean()

2333.3333333333335

## Question 7

Question: List the names of all department managers who have no dependents

SQL code to guide:
``` SQL
SELECT Fname, Lname
FROM EMPLOYEE, DEPARTMENT 
WHERE Ssn=Mgr_ssn
AND NOT EXISTS (SELECT *
                FROM DEPENDENT 
                WHERE Essn = Ssn)
```

In [ ]:
department = pd.read_csv("queries_data/department.csv")
department

,dname,dnumber,mgr_ssn
0,cs,10,2
1,math,20,4
2,bi,30,5


In [ ]:
pd.merge(left=department.loc[~department['mgr_ssn'].isin(pd.unique(dependent['essn']))], right=employee, left_on='mgr_ssn', right_on='ssn')[['fname','lname']]

,fname,lname
0,sergey,paramonov
1,andra,busoniu
